In [54]:
import numpy as np
import matplotlib as mpl

In [55]:
# Question 1
#: the diameter of the wire (x0), 
#the mean of the diameter of coil(x1)
#and the number of active coils (x2)

In [56]:
arr_X = np.array([1, 2, 3])

In [57]:
#implementation of f(x)

In [73]:
def f(x, fCount=[0]):
    print(int(x[0] ** 2)*x[1]*(2 + x[2]))
    fCount[0]+=1
    return (fCount[0])

In [74]:
print(f(arr_X))
print(f(arr_X)[1])
print(f(arr_X))

10


TypeError: 'int' object is not subscriptable

In [60]:
#implementation of g1(x)

In [61]:
def g1(x, g1Count=[0]):
    g1Count[0]+=1
    fun = float(1 - (((x[1]**3)*x[2])/71785*(x[0]**4)))
    if ( fun <= 0) :
        return [True, g1Count[0]]
    else:
        return [False, g1Count[0]]
    
    

In [62]:
print (g1(arr_X))
print (g1(arr_X))
print (g1(arr_X))

[False, 1]
[False, 2]
[False, 3]


In [63]:
#implementation of g2(x)

In [64]:
def g2(x, g2Count=[0]):
    fun = (   (((4*x[1]**2) - (x[0]*x[1]))/(12566*((x[1]*(x[0]**3))-(x[0]**4)))) + (1/(5108*(x[0]**2)))  - 1 )
    g2Count[0]+=1
    if fun <= 0:
        return[True, g2Count[0]]
    else:
        return[False, g2Count[0]]

IndentationError: expected an indented block (<ipython-input-64-2da449554f0a>, line 6)

In [65]:
print (g2(arr_X))
print (g2(arr_X))
print (g2(arr_X))

-0.9986901112011118
4
-0.9986901112011118
5
-0.9986901112011118
6


In [66]:
def g3(x, g3Count=[0]):
    fun = (1 - (140.45*x[0])/((x[1]**2)*x[2]))
    g3Count[0]+=1
    if fun <= 0:   
        return[True, g3Count[0]]
    else:
        return[False, g3Count[0]]
  

In [67]:
print (g3(arr_X))
print (g3(arr_X))
print (g3(arr_X))

-10.704166666666666
1
-10.704166666666666
2
-10.704166666666666
3


In [68]:
def g4(x, g4Count=[0]):
    fun = (  ((x[0] + x[1])/1.5)-1  )
    g4Count[0]+=1
    if fun <= 0:   
        return[True, g4Count[0]]
    else:
        return[False, g4Count[0]]

In [69]:
print (g4(arr_X))
print (g4(arr_X))
print (g4(arr_X))

1.0
1
1.0
2
1.0
3


In [70]:
#Question 2

In [71]:
wire_lower = 0.005 #lower bound for diameter of the wire
wire_upper = 2 #upper bound for diameter of the wire

coil_lower = 0.25 #lower bound for diameter of coil
coil_upper = 1.3 #upper bound for diameter of the coil

coil_num_lower = 2 #lower bound for number of active coils
coil_num_upper = 15 #upper bound for number of active coils

In [72]:
def randomSearch(function, max_iter, lower_bound, upper_bound, fargs=()):
    points_to_evaluate = np.random.uniform(low=lower_bound,
                                         high=upper_bound,
                                         size=(max_iter, len(lower_bound)))
    results = np.array([function(point, *fargs) 
               for point in points_to_evaluate])
    best_ind = np.argmin(results)
    return points_to_evaluate[best_ind], results[best_ind]
    